In [1]:
import os
import sys
os.chdir('/Users/somanathnanda/baba/Covid-19/')

import pandas as pd
pd.options.display.max_columns=None
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

import matplotlib
from plotly.offline import init_notebook_mode, iplot
import warnings

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
warnings.filterwarnings('ignore')
init_notebook_mode(connected=True)

import tensorflow as tf
#tf.get_logger().setLevel('INFO')
#tf.autograph.set_verbosity(2)


from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import time
start = time.process_time()

In [2]:
def plot_timeseries(x_axis,data,name):
    num_cols=data.shape[1]
    go_arr=[]
    for i in range(num_cols):
        go_arr.append(go.Scatter(x=x_axis, y=data.iloc[:,i],name=data.columns[i]))   
    fig=go.Figure(go_arr)
    #fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 1, 0)',})
    fig.update_layout(
    title=name,font=dict(
        family="Courier New, monospace",
        color="#7f7f6a"))
    fig.show()
def plot_lines(df, title='', annotations=None):
    common_kw = dict(x=df.index, mode='lines+markers')
    xaxis = dict(title='Time Steps')
    data=[]
    for c in df.columns:
        data.append(go.Scatter(y=df[c], name=c, **common_kw))
    #data = [go.Scatter(y=df[c], name=c, **common_kw) for c in df.columns]
    layout = dict(title=title, showlegend=True, annotations=annotations, xaxis=xaxis)
    fig=dict(data=data, layout=layout)
    iplot(fig,show_link=True)

In [3]:
confirmed_data_df=pd.read_csv('data/time_series_covid19_confirmed_global.csv')
#recovered_data_df=pd.read_csv('data/time_series_covid19_recovered_global.csv')
death_data_df=pd.read_csv('data/time_series_covid19_deaths_global.csv')

country_pred_df=pd.DataFrame()
model_performance_df=pd.DataFrame()
forecast_df=pd.DataFrame()

def get_time_series_data(country_name):
    grouped_confirmed_df=confirmed_data_df.groupby('Country/Region')
    #grouped_recovered_df=recovered_data_df.groupby('Country/Region')
    grouped_death_df=death_data_df.groupby('Country/Region')
    country_confirmed_df=grouped_confirmed_df.get_group(country_name)
    #country_recovered_df=grouped_recovered_df.get_group(country_name)
    country_death_df=grouped_death_df.get_group(country_name)
    cleaned_confirmed_df=country_confirmed_df.drop(columns=['Province/State','Lat','Long']).sum(axis=0,skipna=True).to_frame()[1:]
    #cleaned_recovered_df=country_recovered_df.drop(columns=['Province/State','Lat','Long']).sum(axis=0,skipna=True).to_frame()[1:]
    cleaned_death_df=country_death_df.drop(columns=['Province/State','Lat','Long']).sum(axis=0,skipna=True).to_frame()[1:]
    ts_df=pd.DataFrame(cleaned_confirmed_df.values,columns=['confirmed_count'])
    #ts_df['recovered_count']=cleaned_recovered_df.values
    ts_df['death_count']=cleaned_death_df.values
    ts_df.index=cleaned_confirmed_df.index
    return ts_df
def plot(ts_df,country_name):
    #plot_timeseries(x_axis=ts_df.index,data=ts_df[['confirmed_count','recovered_count','death_count']],name='count_comparison_'+country_name)
    plot_lines(ts_df,'count_comparison_'+country_name)

# split a univariate sequence into samples
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg
def build_model_and_predict(ts_df, country_name):
    tf.random.set_seed(1234)
    np.random.seed(1)
    time_steps = 5   
    rolling_days = 5
    n_test=1
    n_batch_size = 32
    X=series_to_supervised(ts_df[['confirmed_count']], n_in=time_steps, n_out=rolling_days, dropnan=True)
    train,test=X[:-n_test],X[-n_test:]
    X_train=train.iloc[:,:time_steps]
    Y_train = train.iloc[:,-rolling_days:]
    X_test=test.iloc[:,:time_steps]
    Y_test = test.iloc[:,-rolling_days:]
    
    X_train=X_train.values.astype('float32').reshape((X_train.shape[0], X_train.shape[1], 1))
    Y_train=Y_train.values.astype('float32').reshape((Y_train.shape[0], Y_train.shape[1], 1))
    X_test=X_test.values.astype('float32').reshape((X_test.shape[0], X_test.shape[1], 1))
    Y_test=Y_test.values.astype('float32').reshape((Y_test.shape[0], Y_test.shape[1], 1))
    
    c = [
        ModelCheckpoint(country_name+'.hdf5', save_best_only=True, monitor='val_loss', mode='min', verbose=2, save_freq=1),
        EarlyStopping(monitor='val_loss', min_delta=0, patience=300, verbose=2),
        ReduceLROnPlateau(monitor='val_loss',factor=0.2,patience=5,min_lr=0.00001)
    ]
    model = Sequential()
    model.add(LSTM(100, activation='relu', kernel_initializer='he_normal', input_shape=(time_steps,1)))
    model.add(Dense(50, activation='relu', kernel_initializer='he_normal'))
    model.add(Dense(50, activation='relu', kernel_initializer='he_normal'))
    model.add(Dense(rolling_days))
    # compile the model
    model.compile(optimizer='adam', loss='mse', metrics=['mae','mse','accuracy'])
    # fit the model
    history=model.fit(X_train, Y_train, epochs=2000, batch_size=n_batch_size, verbose=2,callbacks=c,validation_data=(X_test, Y_test),shuffle=False)
    # evaluate the model
    [mse, mae, loss, accuracy]= model.evaluate(X_test, Y_test, batch_size=n_batch_size, verbose=1)
    model_performance=pd.DataFrame({'Country':[country_name],'mae':np.array(mae),'mse':np.array(mse),
                                    'rmse':np.array(np.sqrt(mse)),'accuracy':np.array(accuracy)})
    row = np.asarray(ts_df[-time_steps:].confirmed_count.values.astype('float32')).reshape((1, time_steps, 1))
    yhat=model.predict(row)
    return yhat,model_performance

In [11]:
country_names=['Sweden','India','Ireland','Germany','Australia','Denmark','Iran','Austria',
               'Canada','Singapore','Japan','Portugal','Brazil','US','Norway','United Kingdom']

In [ ]:
for country_name in country_names:
    ts_df=get_time_series_data(country_name)
    yhat, model_performance = build_model_and_predict(ts_df,country_name)
    model_performance_df=model_performance_df.append(model_performance)
    final_date=pd.to_datetime(ts_df[-1:].index)
    pred_df=pd.DataFrame()
    for i in range(yhat.shape[1]):
        pred_df['Country']=[country_name]
        pred_df['pred_date']=final_date+(pd.DateOffset(i+1))
        pred_df['predicted_value']=round(yhat[0][i])
        forecast_df = forecast_df.append(pred_df)
        

Train on 55 samples, validate on 1 samples
Epoch 1/2000
55/55 - 3s - loss: 408499.6576 - mae: 288.1645 - mse: 408499.6250 - accuracy: 0.0364 - val_loss: 3618343.2500 - val_mae: 1768.6237 - val_mse: 3618343.2500 - val_accuracy: 0.0000e+00
Epoch 2/2000
55/55 - 0s - loss: 287359.1892 - mae: 244.5524 - mse: 287359.1875 - accuracy: 0.0364 - val_loss: 2875407.5000 - val_mae: 1465.3834 - val_mse: 2875407.5000 - val_accuracy: 0.0000e+00
Epoch 3/2000
55/55 - 0s - loss: 189859.5739 - mae: 213.9658 - mse: 189859.5781 - accuracy: 0.1273 - val_loss: 1787636.7500 - val_mae: 1133.0520 - val_mse: 1787636.7500 - val_accuracy: 0.0000e+00
Epoch 4/2000
55/55 - 0s - loss: 131022.7218 - mae: 181.5764 - mse: 131022.7188 - accuracy: 0.2000 - val_loss: 1418298.5000 - val_mae: 999.8234 - val_mse: 1418298.5000 - val_accuracy: 0.0000e+00
Epoch 5/2000
55/55 - 0s - loss: 94376.8156 - mae: 148.3064 - mse: 94376.8203 - accuracy: 0.2727 - val_loss: 499618.4062 - val_mae: 639.1378 - val_mse: 499618.4062 - val_accuracy:

55/55 - 0s - loss: 36301.1432 - mae: 91.9013 - mse: 36301.1445 - accuracy: 0.3273 - val_loss: 221406.4531 - val_mae: 439.3256 - val_mse: 221406.4531 - val_accuracy: 0.0000e+00
Epoch 25/2000
55/55 - 0s - loss: 36161.5583 - mae: 91.8521 - mse: 36161.5586 - accuracy: 0.3273 - val_loss: 220312.5469 - val_mae: 438.3835 - val_mse: 220312.5469 - val_accuracy: 0.0000e+00
Epoch 26/2000
55/55 - 0s - loss: 35993.9094 - mae: 91.8711 - mse: 35993.9102 - accuracy: 0.3273 - val_loss: 219199.7188 - val_mae: 437.4161 - val_mse: 219199.7188 - val_accuracy: 0.0000e+00
Epoch 27/2000
55/55 - 0s - loss: 35887.0342 - mae: 91.9712 - mse: 35887.0352 - accuracy: 0.3273 - val_loss: 218057.1250 - val_mae: 436.4139 - val_mse: 218057.1250 - val_accuracy: 0.0000e+00
Epoch 28/2000
55/55 - 0s - loss: 35807.0728 - mae: 92.0281 - mse: 35807.0703 - accuracy: 0.3273 - val_loss: 216877.2969 - val_mae: 435.3690 - val_mse: 216877.2969 - val_accuracy: 0.0000e+00
Epoch 29/2000
55/55 - 0s - loss: 35732.6785 - mae: 91.9934 - mse

55/55 - 0s - loss: 34080.3206 - mae: 90.3999 - mse: 34080.3203 - accuracy: 0.3273 - val_loss: 191290.9688 - val_mae: 411.2088 - val_mse: 191290.9688 - val_accuracy: 1.0000
Epoch 49/2000
55/55 - 0s - loss: 33998.6408 - mae: 90.3112 - mse: 33998.6406 - accuracy: 0.3273 - val_loss: 190037.2031 - val_mae: 409.9595 - val_mse: 190037.2031 - val_accuracy: 1.0000
Epoch 50/2000
55/55 - 0s - loss: 33916.9121 - mae: 90.2368 - mse: 33916.9102 - accuracy: 0.3273 - val_loss: 188793.1094 - val_mae: 408.7134 - val_mse: 188793.1094 - val_accuracy: 1.0000
Epoch 51/2000
55/55 - 0s - loss: 33835.3009 - mae: 90.1734 - mse: 33835.3008 - accuracy: 0.3273 - val_loss: 187557.0312 - val_mae: 407.4691 - val_mse: 187557.0312 - val_accuracy: 1.0000
Epoch 52/2000
55/55 - 0s - loss: 33754.5098 - mae: 90.1109 - mse: 33754.5117 - accuracy: 0.3273 - val_loss: 186325.2500 - val_mae: 406.2225 - val_mse: 186325.2500 - val_accuracy: 1.0000
Epoch 53/2000
55/55 - 0s - loss: 33674.2971 - mae: 90.0408 - mse: 33674.2969 - accur

55/55 - 0s - loss: 32242.0055 - mae: 88.5914 - mse: 32242.0039 - accuracy: 0.3636 - val_loss: 160412.8438 - val_mae: 373.5167 - val_mse: 160412.8438 - val_accuracy: 1.0000
Epoch 73/2000
55/55 - 0s - loss: 32171.3000 - mae: 88.5181 - mse: 32171.3008 - accuracy: 0.3636 - val_loss: 158022.1094 - val_mae: 362.0703 - val_mse: 158022.1094 - val_accuracy: 0.0000e+00
Epoch 74/2000
55/55 - 0s - loss: 32101.1955 - mae: 88.4442 - mse: 32101.1953 - accuracy: 0.3636 - val_loss: 161621.1875 - val_mae: 330.2789 - val_mse: 161621.1875 - val_accuracy: 0.0000e+00
Epoch 75/2000
55/55 - 0s - loss: 32031.5844 - mae: 88.3689 - mse: 32031.5859 - accuracy: 0.3636 - val_loss: 183309.7500 - val_mae: 334.5396 - val_mse: 183309.7500 - val_accuracy: 0.0000e+00
Epoch 76/2000
55/55 - 0s - loss: 31962.4111 - mae: 88.2932 - mse: 31962.4121 - accuracy: 0.3636 - val_loss: 239373.8438 - val_mae: 411.3170 - val_mse: 239373.8438 - val_accuracy: 0.0000e+00
Epoch 77/2000
55/55 - 0s - loss: 31893.7442 - mae: 88.2181 - mse: 31

Epoch 96/2000
55/55 - 0s - loss: 30672.2545 - mae: 86.8188 - mse: 30672.2539 - accuracy: 0.3455 - val_loss: 461562.0938 - val_mae: 552.2941 - val_mse: 461562.0938 - val_accuracy: 0.0000e+00
Epoch 97/2000
55/55 - 0s - loss: 30612.3825 - mae: 86.7459 - mse: 30612.3809 - accuracy: 0.3455 - val_loss: 459898.9062 - val_mae: 551.2140 - val_mse: 459898.9062 - val_accuracy: 0.0000e+00
Epoch 98/2000
55/55 - 0s - loss: 30552.8436 - mae: 86.6735 - mse: 30552.8457 - accuracy: 0.3455 - val_loss: 458242.9062 - val_mae: 550.1436 - val_mse: 458242.9062 - val_accuracy: 0.0000e+00
Epoch 99/2000
55/55 - 0s - loss: 30493.7556 - mae: 86.6016 - mse: 30493.7559 - accuracy: 0.3455 - val_loss: 456521.4375 - val_mae: 549.0486 - val_mse: 456521.4375 - val_accuracy: 0.0000e+00
Epoch 100/2000
55/55 - 0s - loss: 30435.0074 - mae: 86.5295 - mse: 30435.0098 - accuracy: 0.3455 - val_loss: 454707.5000 - val_mae: 547.9166 - val_mse: 454707.5000 - val_accuracy: 0.0000e+00
Epoch 101/2000
55/55 - 0s - loss: 30376.5401 - ma

...

55/55 - 0s - loss: 1860304.0977 - mae: 755.4061 - mse: 1860304.1250 - accuracy: 0.5455 - val_loss: 65052468.0000 - val_mae: 7625.6533 - val_mse: 65052468.0000 - val_accuracy: 0.0000e+00
Epoch 199/2000
55/55 - 0s - loss: 1858762.8091 - mae: 755.1389 - mse: 1858762.8750 - accuracy: 0.5455 - val_loss: 65016296.0000 - val_mae: 7623.6172 - val_mse: 65016296.0000 - val_accuracy: 0.0000e+00
Epoch 200/2000
55/55 - 0s - loss: 1857035.6841 - mae: 754.8401 - mse: 1857035.6250 - accuracy: 0.5455 - val_loss: 64981108.0000 - val_mae: 7621.6030 - val_mse: 64981108.0000 - val_accuracy: 0.0000e+00
Epoch 201/2000
55/55 - 0s - loss: 1855172.2909 - mae: 754.5428 - mse: 1855172.3750 - accuracy: 0.5455 - val_loss: 64947040.0000 - val_mae: 7619.6304 - val_mse: 64947040.0000 - val_accuracy: 0.0000e+00
Epoch 202/2000
55/55 - 0s - loss: 1853874.1045 - mae: 754.3483 - mse: 1853874.0000 - accuracy: 0.5455 - val_loss: 64914056.0000 - val_mae: 7617.6924 - val_mse: 64914056.0000 - val_accuracy: 0.0000e+00
Epoch 203/

55/55 - 0s - loss: 1830713.3511 - mae: 750.3276 - mse: 1830713.2500 - accuracy: 0.5455 - val_loss: 64468736.0000 - val_mae: 7586.4141 - val_mse: 64468736.0000 - val_accuracy: 0.0000e+00
Epoch 222/2000
55/55 - 0s - loss: 1829514.0841 - mae: 750.1011 - mse: 1829514.0000 - accuracy: 0.5455 - val_loss: 64453992.0000 - val_mae: 7585.0264 - val_mse: 64453992.0000 - val_accuracy: 0.0000e+00
Epoch 223/2000
55/55 - 0s - loss: 1828313.9795 - mae: 749.8727 - mse: 1828313.8750 - accuracy: 0.5455 - val_loss: 64440012.0000 - val_mae: 7583.6626 - val_mse: 64440012.0000 - val_accuracy: 0.0000e+00
Epoch 224/2000
55/55 - 0s - loss: 1827114.1182 - mae: 749.6423 - mse: 1827114.0000 - accuracy: 0.5455 - val_loss: 64426824.0000 - val_mae: 7582.3213 - val_mse: 64426824.0000 - val_accuracy: 0.0000e+00
Epoch 225/2000
55/55 - 0s - loss: 1825912.9886 - mae: 749.4097 - mse: 1825912.8750 - accuracy: 0.5455 - val_loss: 64415200.0000 - val_mae: 7581.0234 - val_mse: 64415200.0000 - val_accuracy: 0.0000e+00
Epoch 226/

55/55 - 0s - loss: 1803296.6159 - mae: 744.7525 - mse: 1803296.6250 - accuracy: 0.5455 - val_loss: 64393432.0000 - val_mae: 7562.6392 - val_mse: 64393432.0000 - val_accuracy: 0.0000e+00
Epoch 245/2000
55/55 - 0s - loss: 1802104.1011 - mae: 744.5161 - mse: 1802104.1250 - accuracy: 0.5455 - val_loss: 64401704.0000 - val_mae: 7561.9297 - val_mse: 64401704.0000 - val_accuracy: 0.0000e+00
Epoch 246/2000
55/55 - 0s - loss: 1800910.1000 - mae: 744.2782 - mse: 1800910.0000 - accuracy: 0.5455 - val_loss: 64410792.0000 - val_mae: 7561.2358 - val_mse: 64410792.0000 - val_accuracy: 0.0000e+00
Epoch 247/2000
55/55 - 0s - loss: 1799716.2875 - mae: 744.0392 - mse: 1799716.3750 - accuracy: 0.5455 - val_loss: 64420716.0000 - val_mae: 7560.5649 - val_mse: 64420716.0000 - val_accuracy: 0.0000e+00
Epoch 248/2000
55/55 - 0s - loss: 1798521.6216 - mae: 743.7990 - mse: 1798521.5000 - accuracy: 0.5455 - val_loss: 64431348.0000 - val_mae: 7559.9072 - val_mse: 64431348.0000 - val_accuracy: 0.0000e+00
Epoch 249/

55/55 - 0s - loss: 1776214.5284 - mae: 739.0939 - mse: 1776214.5000 - accuracy: 0.5455 - val_loss: 64735052.0000 - val_mae: 7549.2471 - val_mse: 64735052.0000 - val_accuracy: 0.0000e+00
Epoch 268/2000
55/55 - 0s - loss: 1775073.4455 - mae: 738.8434 - mse: 1775073.5000 - accuracy: 0.5455 - val_loss: 64756952.0000 - val_mae: 7548.8320 - val_mse: 64756952.0000 - val_accuracy: 0.0000e+00
Epoch 269/2000
55/55 - 0s - loss: 1773931.2818 - mae: 738.5936 - mse: 1773931.3750 - accuracy: 0.5455 - val_loss: 64780052.0000 - val_mae: 7548.4517 - val_mse: 64780052.0000 - val_accuracy: 0.0000e+00
Epoch 270/2000
55/55 - 0s - loss: 1772789.3114 - mae: 738.3438 - mse: 1772789.2500 - accuracy: 0.5455 - val_loss: 64804264.0000 - val_mae: 7548.1040 - val_mse: 64804264.0000 - val_accuracy: 0.0000e+00
Epoch 271/2000
55/55 - 0s - loss: 1771649.8443 - mae: 738.0950 - mse: 1771649.8750 - accuracy: 0.5455 - val_loss: 64829300.0000 - val_mae: 7547.7627 - val_mse: 64829300.0000 - val_accuracy: 0.0000e+00
Epoch 272/

55/55 - 0s - loss: 1750602.7523 - mae: 733.5408 - mse: 1750602.7500 - accuracy: 0.5636 - val_loss: 65430008.0000 - val_mae: 7542.7588 - val_mse: 65430008.0000 - val_accuracy: 0.0000e+00
Epoch 291/2000
55/55 - 0s - loss: 1749524.4148 - mae: 733.3051 - mse: 1749524.5000 - accuracy: 0.5636 - val_loss: 65466496.0000 - val_mae: 7542.5732 - val_mse: 65466496.0000 - val_accuracy: 0.0000e+00
Epoch 292/2000
55/55 - 0s - loss: 1748443.2773 - mae: 733.0701 - mse: 1748443.3750 - accuracy: 0.5636 - val_loss: 65503852.0000 - val_mae: 7542.4404 - val_mse: 65503852.0000 - val_accuracy: 0.0000e+00
Epoch 293/2000
55/55 - 0s - loss: 1747362.8818 - mae: 732.8359 - mse: 1747362.8750 - accuracy: 0.5636 - val_loss: 65541736.0000 - val_mae: 7542.3096 - val_mse: 65541736.0000 - val_accuracy: 0.0000e+00
Epoch 294/2000


In [ ]:
for country_name in country_names:
    ts_df=get_time_series_data(country_name)
    df_1=ts_df[['confirmed_count']]
    df_1['date']=pd.to_datetime(df_1.index)
    df_1['date']=df_1['date'].dt.strftime('%Y-%m-%d')
    df_1.reset_index(drop=True, inplace=True)
    df_1.index=df_1['date']
    
    grouped_country_df=forecast_df.groupby('Country').get_group(country_name)
    grouped_country_df['pred_date']=grouped_country_df['pred_date'].dt.strftime('%Y-%m-%d')
    grouped_country_df.index=grouped_country_df.pred_date
    df_2=grouped_country_df['predicted_value']
    df_all=pd.merge(df_1,df_2,how='outer', left_index=True,right_index=True)
    
    plot(df_all,country_name)
    

In [ ]:
forecast_df.to_html()

#### Compare the model accuracy for each country

In [ ]:
model_performance_df.index=model_performance_df['Country']
model_performance_df=model_performance_df.drop(columns=['Country'])
plot_lines(model_performance_df)

In [ ]:
print('Total Time taken : '+str((time.process_time() - start)/60)+' mins')